In [2]:
import subprocess

command = ['yt-dlp', '--extract-audio', '--audio-format', 'mp3', 'https://www.youtube.com/watch?v=VLiQm9Z6wFw']

In [3]:
import os

In [4]:
os.chdir('../data/')
print(os.getcwd())

/Users/samsonbakos/Documents/Job_Materials/Application_Materials/Scotiabank/SnoopChat/data


In [5]:
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("Output:", result.stdout)
print("Error:", result.stderr)

Output: [youtube] Extracting URL: https://www.youtube.com/watch?v=VLiQm9Z6wFw
[youtube] VLiQm9Z6wFw: Downloading webpage
[youtube] VLiQm9Z6wFw: Downloading ios player API JSON
[youtube] VLiQm9Z6wFw: Downloading android player API JSON
[youtube] VLiQm9Z6wFw: Downloading m3u8 information
[info] VLiQm9Z6wFw: Downloading 1 format(s): 251
[download] Oscar Winner Kathy Bates Gets Disjointed With Snoop Dogg ｜ GGN NEWS [VLiQm9Z6wFw].webm has already been downloaded

[download] 100% of   17.82MiB

Error: WARNING: [youtube] Skipping player responses from android clients (got player responses for video "aQvGIIdgFDM" instead of "VLiQm9Z6wFw")
ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location



In [6]:
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("Output:", result.stdout)
print("Error:", result.stderr)

Output: [youtube] Extracting URL: https://www.youtube.com/watch?v=VLiQm9Z6wFw
[youtube] VLiQm9Z6wFw: Downloading webpage
[youtube] VLiQm9Z6wFw: Downloading ios player API JSON
[youtube] VLiQm9Z6wFw: Downloading android player API JSON
[youtube] VLiQm9Z6wFw: Downloading m3u8 information
[info] VLiQm9Z6wFw: Downloading 1 format(s): 251
[download] Oscar Winner Kathy Bates Gets Disjointed With Snoop Dogg ｜ GGN NEWS [VLiQm9Z6wFw].webm has already been downloaded

[download] 100% of   17.82MiB

Error: WARNING: [youtube] Skipping player responses from android clients (got player responses for video "aQvGIIdgFDM" instead of "VLiQm9Z6wFw")
ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location



In [7]:
from google.cloud import speech_v1p1beta1 as speech

client = speech.SpeechClient()

audio = speech.RecognitionAudio(uri="gs://snoopchat/Oscar Winner Kathy Bates Gets Disjointed With Snoop Dogg ｜ GGN NEWS [VLiQm9Z6wFw].webm")
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.WEBM_OPUS,
    sample_rate_hertz=48000,  
    language_code="en-US",
    enable_speaker_diarization=True,
    diarization_speaker_count=2,
    audio_channel_count=2,  
    enable_separate_recognition_per_channel=False  
)

operation = client.long_running_recognize(config=config, audio=audio)

print("Waiting for operation to complete...")
response = operation.result(timeout= None)



Waiting for operation to complete...


In [8]:

speaker_word_counts = {}

for result in response.results:
    alternative = result.alternatives[0]
    for word_info in alternative.words:
     
        speaker_tag = word_info.speaker_tag
        if speaker_tag in speaker_word_counts:
            speaker_word_counts[speaker_tag] += 1
        else:
            speaker_word_counts[speaker_tag] = 1

for speaker_tag, count in speaker_word_counts.items():
    print(f"Speaker {speaker_tag}: {count} words")

Speaker 0: 2626 words
Speaker 2: 2264 words
Speaker 1: 362 words


In [9]:
speaker_0_segments = []  

for result in response.results:
    for word_info in result.alternatives[0].words:
        if word_info.speaker_tag == 0:
            start_time = word_info.start_time.total_seconds()
            end_time = word_info.end_time.total_seconds()
            speaker_0_segments.append((start_time, end_time))

merged_segments = []
for start, end in sorted(speaker_0_segments):
    if merged_segments and start <= merged_segments[-1][1]:
        merged_segments[-1][1] = max(merged_segments[-1][1], end) 
    else:
        merged_segments.append([start, end])

speaker_0_segments = [(start, end) for start, end in merged_segments]


In [10]:
os.getcwd()

'/Users/samsonbakos/Documents/Job_Materials/Application_Materials/Scotiabank/SnoopChat/data'

In [41]:
import os

output_dir = "rawmp3/"
source_file = "Oscar Winner Kathy Bates Gets Disjointed With Snoop Dogg ｜ GGN NEWS [VLiQm9Z6wFw].webm"
concat_list_path = os.path.join(output_dir, "concat_list.txt")

# ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

ffmpeg_path = "/opt/homebrew/bin/ffmpeg"

# create a list to store segment file paths
segment_files = []

# generate segments and add file paths to the list
for idx, (start, end) in enumerate(speaker_0_segments, 1):
    segment_file = os.path.join(output_dir, f"speaker_0_segment_{idx}.mp3")
    segment_files.append(segment_file)
    # generate ffmpeg command for extracting each segment
    ffmpeg_command = f'"{ffmpeg_path}" -y -i "{source_file}" -ss {start} -to {end} -codec:a libmp3lame -q:a 4 "{segment_file}"'
    os.system(ffmpeg_command)

# write file paths to the concat list
with open(concat_list_path, 'w') as list_file:
    for segment_file in segment_files:
        list_file.write(f"file '{segment_file}'\n")




ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopen

In [29]:
os.chdir('../training/')

In [30]:
os.getcwd()

'/Users/samsonbakos/Documents/Job_Materials/Application_Materials/Scotiabank/SnoopChat/training'

In [37]:
import os

# Define absolute path for the output directory
output_dir = "/Users/samsonbakos/Documents/Job_Materials/Application_Materials/Scotiabank/SnoopChat/data/rawmp3/"

# Define absolute path for the concat list
concat_list_path = os.path.join(output_dir, "concat_list.txt")

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

ffmpeg_path = "/opt/homebrew/bin/ffmpeg"

# Define absolute path for the output file
output_file = os.path.join(output_dir, "speaker_0_full.mp3")

print(output_dir)
print(concat_list_path)
print(output_file)

 # Write file paths to the concat list with absolute paths
with open(concat_list_path, 'w') as list_file:
    for segment_file in segment_files:
        abs_segment_file = os.path.abspath(segment_file)
        list_file.write(f"file '{abs_segment_file}'\n")

# Generate the ffmpeg concat command
ffmpeg_concat_command = f'"{ffmpeg_path}" -f concat -safe 0 -i "{concat_list_path}" -c copy "{output_file}"'

# Execute the ffmpeg command
os.system(ffmpeg_concat_command)


/Users/samsonbakos/Documents/Job_Materials/Application_Materials/Scotiabank/SnoopChat/data/rawmp3/
/Users/samsonbakos/Documents/Job_Materials/Application_Materials/Scotiabank/SnoopChat/data/rawmp3/concat_list.txt
/Users/samsonbakos/Documents/Job_Materials/Application_Materials/Scotiabank/SnoopChat/data/rawmp3/speaker_0_full.mp3


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopen

65024

In [13]:
os.getcwd()

'/Users/samsonbakos/Documents/Job_Materials/Application_Materials/Scotiabank/SnoopChat/data'

In [14]:
!ls

Oscar Winner Kathy Bates Gets Disjointed With Snoop Dogg ｜ GGN NEWS [VLiQm9Z6wFw].webm
rawmp3
transcript.txt
transcript_for_fine_tuning.jsonl
